In [1]:
import numpy as np
import scipy as scp
import scipy.stats as ss
import sklearn.metrics as sm

from scipy.integrate import quad

import matplotlib.pyplot as plt
import scipy.special as scps
from statsmodels.graphics.gofplots import qqplot
from scipy.linalg import cholesky
from functools import partial
from scipy.optimize import minimize, brute, fmin
from IPython.display import display
import sympy; sympy.init_printing()


def display_matrix(m):
    display(sympy.Matrix(m))

In [2]:
def cf_Heston_schoutens(u, t, v0, mu, kappa, theta, rho, sigma):
    """
    Heston characteristic function as proposed by Schoutens (2004)
    """
    xi = kappa - sigma*rho*u*1j
    d = np.sqrt( xi**2 + sigma**2 * (u**2 + 1j*u) )
    g1 = (xi+d)/(xi-d)
    g2 = 1/g1
    cf = np.exp( 1j*u*mu*t + (kappa*theta)/(sigma**2) * ( (xi-d)*t - 2*np.log( (1-g2*np.exp(-d*t))/(1-g2) ))\
              + (v0/sigma**2)*(xi-d) * (1-np.exp(-d*t))/(1-g2*np.exp(-d*t)) )
    return cf

In [3]:
def Q1(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the stock numeraire.
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( (np.exp(-u*k*1j) / (u*1j)) * 
                                  cf(u-1j) / cf(-1.0000000000001j) )  
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=500 )[0]

def Q2(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the money market numeraire
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( np.exp(-u*k*1j) /(u*1j) * cf(u) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=500 )[0]


In [4]:
mu = 0.05                                          # drift
rho = -0.8                                         # correlation coefficient
kappa = 3                                          # mean reversion coefficient
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance
T = 15                                             # Time to mat
r = 0.05                                           # risk-free rate
K = 100                                            # Stike  
v0 = 0.08                                          # spot variance
S0 = 100                                           # spot stock price 
k = np.log(K/S0) 

par = (mu, theta, sigma, kappa, rho)

cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 

limit_max = 1000      # right limit in the integration                
call = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T)* Q2(k, cf_Heston_schoutens_valuation, limit_max)
print(call)

65.27786862734618


In [5]:
def call_price(S0, K, T, v0, r, par): #BUGGY
    limit_max = 1000
    mu, theta, sigma, kappa, rho = par
    cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 
    model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
    return model_price

In [13]:
call_price(S0=S0, K=K, T=T, v0=v0, r=r, par=par)

In [59]:
def call_loss_function(par, verbose = 1, **kwargs):    
    global i 
    mu, theta, sigma, kappa, rho = par
    
    # Parse parameters from **kwargs
    S0 = kwargs['S0']
    K =  kwargs['K']
    T =  kwargs['T']
    r =  kwargs['r']
    v0 = kwargs['v0']
    market_price = kwargs['market_price']
    
    # log of strike price divided by S0
    k = np.log(K/S0)
    
    try:
        limit_max = kwargs['limit_max']
    except:
        limit_max = 1000
    
    cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 
    
    model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) \
                        - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
    
    # Absolute difference between market and model price
    loss = np.abs((market_price - model_price))    
 
    if verbose == 1:
        if i==0:
            print('  mu      theta   sigma   kappa    rho    loss')
        if i % 25 == 0:
            print('%7.3f %7.3f %7.3f %7.3f %7.3f' %(mu, theta, sigma, kappa, rho), '%7.3f' %loss)
   
    i+=1
    
    return loss

In [49]:
i=0

In [60]:
i=0
# mu, theta, sigma, kappa, rho
mu = 0.05                                          # drift
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance
kappa = 3                                          # mean reversion coefficient
rho = -0.8                                         # correlation coefficient
T = 15                                             # Time to mat
r = 0.05                                           # risk-free rate
K = 100                                            # Stike  
v0 = 0.08                                          # spot variance
S0 = 100                                           # spot stock price 
k = np.log(K/S0) 

# Check if the prices match
limit_max = 1000      # right limit in the integration                
call = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T)* Q2(k, cf_Heston_schoutens_valuation, limit_max)
print('Call Price from equation 1:',call)

par = (mu, theta, sigma, kappa, rho)
x = call_price(S0=S0, K=K, T=T, v0=v0, r=r, par=par)
print('Call Price from equation 2:',x)

loss = call_loss_function(par, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)
print('loss:',loss)

Call Price from equation 1: 65.27786862734618
Call Price from equation 2: 65.27786862734618
  mu      theta   sigma   kappa    rho    loss
  0.050   0.100   0.250   3.000  -0.800   0.000
loss: 0.0


In [61]:
%%time
i=0
# init_par = np.array([mu, theta, sigma, kappa, rho]) + np.random.normal(0,.01,5)
# init_par = np.random.normal(0,.01,5)

init_par = (np.random.uniform(.05,.95), #mu
            np.random.uniform(.5,5),    #theta
            np.random.uniform(.05,.95), #sigma
            np.random.uniform(.5,5),    #kappa
            np.random.uniform(-.9,-.1)) #rho

call_loss_function_to_optimize = partial(call_loss_function, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)

res = minimize(call_loss_function_to_optimize, init_par, method='nelder-mead',
              bounds = ((.05,.95),(.5,5),(.05,.95),(.5,5),(-.9,-.1)),
               options={'xtol': 1e-10, 'disp': True})
print('init par:', list(init_par))
print('target:', np.array([mu, theta, sigma, kappa, rho]))
print('result:',res.x)
print('MSE init par vs target:', sm.mean_squared_error(init_par, np.array([mu, theta, sigma, kappa, rho])))
print('MSE result vs target:', sm.mean_squared_error(res.x, np.array([mu, theta, sigma, kappa, rho])))

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:518: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  RuntimeWarning)


  mu      theta   sigma   kappa    rho    loss
  0.305   2.054   0.854   4.788  -0.443  11.086
  0.274   2.019   0.941   4.888  -0.476  11.077
  0.165   1.832   1.026   5.378  -0.687  10.984
  0.106   1.751   1.106   5.925  -0.784  10.852
  0.072   1.696   1.198   6.443  -0.862  10.714
 -0.012   1.560   1.416   7.765  -1.040  33.114
 -0.101   1.418   1.747   9.427  -1.230  31.093
 -0.113   1.399   1.797   9.668  -1.257   7.272
 -0.129   1.371   1.855   9.950  -1.293   6.459
 -0.151   1.334   1.948  10.378  -1.343   5.074
 -0.183   1.278   2.062  10.910  -1.414  25.857
 -0.191   1.261   2.068  10.961  -1.433   1.571
 -0.207   1.228   2.084  11.082  -1.473  23.061
 -0.204   1.234   2.080  11.055  -1.466   0.017
 -0.204   1.234   2.082  11.062  -1.465   0.002
 -0.204   1.234   2.081  11.060  -1.465   0.001
 -0.204   1.234   2.081  11.060  -1.465   0.000
 -0.204   1.234   2.081  11.060  -1.465   0.000
 -0.204   1.234   2.081  11.060  -1.465   0.000
 -0.204   1.234   2.081  11.060  -1.465  

In [62]:
call_price(S0=S0, K=K, T=T, v0=v0, r=r, par=res.x)

In [42]:
init_par = (np.random.uniform(.05,.95), #mu
            np.random.uniform(.5,5),    #theta
            np.random.uniform(.05,.95), #sigma
            np.random.uniform(.5,5),    #kappa
            np.random.uniform(-.9,-.1)) #rho

# p0 = brute(call_loss_function_to_optimize,
#            ((2.5, 10.6, 5.0),  # kappa_v
#             (0.01, 0.041, 0.01),  # theta_v
#             (0.05, 0.251, 0.1),  # sigma_v
#             (-0.75, 0.01, 0.25),  # rho
#             (0.01, 0.031, 0.01)),  # v0
#            finish=None)

p0 = brute(call_loss_function_to_optimize,
           (
               (0.01, 0.031, 0.01), #mu
               (0.01, 0.041, 0.01), #theta
               (0.05, 0.251, 0.1),  #sigma
               (2.5, 10.6, 5.0),    #kappa
               (-0.75, 0.01, 0.25)  #rho
           ),
           finish=None)

# second run with local, convex minimization
# (dig deeper where promising)
opt = fmin(call_loss_function_to_optimize, p0,
           xtol=0.000001, ftol=0.000001,
           maxiter=750, maxfun=900)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in cdouble_scalars
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in cdouble_scalars
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in cdouble_scalars
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


ValueError: cannot convert float NaN to integer

In [29]:
call_price(opt)

TypeError: call_price() missing 5 required positional arguments: 'K', 'T', 'v0', 'r', and 'par'